In [ ]:
# Decide whether or not running on Kaggle
import os
KAGGLE_RUN = True
if os.path.isdir('/home/ubuntu'):
    KAGGLE_RUN = False

import sys
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
import lightgbm as lgb
import datatable as dt
import json
import time
import pickle
import dill

if not KAGGLE_RUN:
    sys.path.insert(0, './input')
    import local_work
    
import riiideducation

# Boto3 Setup and Download Files
if not KAGGLE_RUN:
    local_work = local_work.local_work()
    local_work.download_riiid_files()

import random
import os


In [ ]:
def get_questions(KAGGLE_RUN):
    # Load questions
    if not KAGGLE_RUN:
        questions_df = local_work.get_questions_data()
        question_cmnts = pd.read_csv('./input/question_cmnts_v5.csv')
    else:   
        questions_df = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
        question_cmnts = pd.read_csv('../input/question-cmnts-v5/question_cmnts_v5.csv')
        
    # Add Communities to questions
    question_cmnts.columns = ['question_id', 'community']
    questions_df = questions_df.merge(question_cmnts, on = 'question_id', how = 'left')
    
    # Get tags of questions
    inv_rows = questions_df[questions_df['tags'].apply(type) == float].index
    questions_df.at[inv_rows, 'tags'] = ''
    questions_df['tags'] = questions_df['tags'].apply(lambda x: np.array(x.split()).astype(int))
    questions_df['n_tags'] = questions_df['tags'].str.len()
    questions_df.loc[questions_df.n_tags == 0, 'n_tags'] = 1
    questions_df['tag_1'] = questions_df['tags'].map(lambda x: x[0] if len(x) > 0 else np.nan)
    questions_df['tag_2'] = questions_df['tags'].map(lambda x: x[1] if len(x) > 1 else np.nan)
    questions_df['tag_3'] = questions_df['tags'].map(lambda x: x[2] if len(x) > 2 else np.nan)
        
    # Change questions dtypes
    questions_df['part'] = questions_df['part'].astype(np.int32)
    questions_df['bundle_id'] = questions_df['bundle_id'].astype(np.int32)
    questions_df['community'] = questions_df['community'].astype(np.int8)
    
    return(questions_df)

def get_cond_probs(KAGGLE_RUN):
    # Load conditional probabilities
    if KAGGLE_RUN:
        with open('../input/finalmodels/cond_probs.pickle', 'rb') as handle:
            cond_probs =  pickle.load(handle)
    else:
        with open('./output/cond_probs.pickle', 'rb') as handle:
            cond_probs =  pickle.load(handle)
    return(cond_probs)

# Funcion for user stats with loops
def add_features(df, features_dicts, cond_probs, KAGGLE_RUN, is_inference):
    
            
    # -----------------------------------------------------------------------
    # Get feature dicts
    d_answered_correctly_u_count = features_dicts['d_answered_correctly_u_count']
    d_answered_correctly_u_sum = features_dicts['d_answered_correctly_u_sum']
    d_answered_correctly_prev10_u = features_dicts['d_answered_correctly_prev10_u']
    d_elapsed_time_u_sum = features_dicts['d_elapsed_time_u_sum']
    d_explanation_u_sum = features_dicts['d_explanation_u_sum']
    d_answered_correctly_q_count = features_dicts['d_answered_correctly_q_count']
    d_answered_correctly_q_sum = features_dicts['d_answered_correctly_q_sum']
    d_elapsed_time_q_sum = features_dicts['d_elapsed_time_q_sum']
    d_explanation_ac_q_count = features_dicts['d_explanation_ac_q_count']
    d_answered_correctly_uq = features_dicts['d_answered_correctly_uq']
    d_timestamp_u = features_dicts['d_timestamp_u']
    d_timestamp_u_incorrect = features_dicts['d_timestamp_u_incorrect']
    d_answered_correctly_up_sum = features_dicts['d_answered_correctly_up_sum']
    d_answered_correctly_up_count = features_dicts['d_answered_correctly_up_count']
    d_answered_correctly_uc_count = features_dicts['d_answered_correctly_uc_count']
    d_answered_correctly_uc_sum = features_dicts['d_answered_correctly_uc_sum']
    d_answered_correctly_ut_count = features_dicts['d_answered_correctly_ut_count']
    d_answered_correctly_ut_sum = features_dicts['d_answered_correctly_ut_sum']
    d_lectures_u_count = features_dicts['d_lectures_u_count']
    d_uq_prev = features_dicts['d_uq_prev']
    
    
    # -----------------------------------------------------------------------
    # Client features
    answered_correctly_u_avg = np.zeros(len(df), dtype = np.float32)
    answered_correctly_prev10_u_avg = np.zeros(len(df), dtype = np.float32)
    answered_correctly_prev5_u_avg = np.zeros(len(df), dtype = np.float32)
    elapsed_time_u_avg = np.zeros(len(df), dtype = np.float32)
    explanation_u_avg = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_1 = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_2 = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_3 = np.zeros(len(df), dtype = np.float32)
    timestamp_u_incorrect_recency = np.zeros(len(df), dtype = np.float32)
    
    lectures_u_count = np.zeros(len(df), dtype = np.int32)
    # -----------------------------------------------------------------------
    # Question features
    answered_correctly_q_avg = np.zeros(len(df), dtype = np.float32)
    elapsed_time_q_avg = np.zeros(len(df), dtype = np.float32)
    explanation_q_avg = np.zeros(len(df), dtype = np.float32)
    # -----------------------------------------------------------------------
    # User Question
    answered_correctly_uq_count = np.zeros(len(df), dtype = np.int32)
    # -----------------------------------------------------------------------
    # User Part
    answered_correctly_up_count = np.zeros(len(df), dtype = np.int32)
    answered_correctly_up_avg = np.zeros(len(df), dtype = np.float32)
    # -----------------------------------------------------------------------
    # User Community
    answered_correctly_uc_count = np.zeros(len(df), dtype = np.int32)
    answered_correctly_uc_avg = np.zeros(len(df), dtype = np.float32)    
    # -----------------------------------------------------------------------
    # User Tags
    answered_correctly_ut_cum_count = np.zeros(len(df), dtype = np.int32)
    answered_correctly_ut_cum_sum = np.zeros(len(df), dtype = np.int)
    answered_correctly_ut_cum_avg = np.zeros(len(df), dtype = np.float)
    # -----------------------------------------------------------------------
    # User Questions
    cond_uq_prob = np.zeros(len(df), dtype = np.float)
    
    # -----------------------------------------------------------------------
    
    
    
    DF_NROW = df.shape[0]
    for num, row in enumerate(df[['user_id'
                                  ,'answered_correctly'
                                  ,'content_id'
                                  ,'prior_question_elapsed_time'
                                  ,'prior_question_had_explanation'
                                  ,'timestamp'
                                  ,'part'
                                  ,'community'
                                  ,'tags' 
                                  ,'content_type_id' # 9
                                 ]].values):
        
        if DF_NROW > 5000000 and not KAGGLE_RUN:
            if num % 10000000 == 0:
                print('Progress: ' + str(round(num/DF_NROW, 3)))
                
        if row[9] == 1:
            # Lecture updates
            # ------------------------------------------------------------------
            d_lectures_u_count[row[0]] += 1
            
        else:
            # Client features assignation
            # ------------------------------------------------------------------
            if d_answered_correctly_u_count[row[0]] != 0:
                answered_correctly_u_avg[num] = d_answered_correctly_u_sum[row[0]] /\
                                                    d_answered_correctly_u_count[row[0]]
                elapsed_time_u_avg[num] = d_elapsed_time_u_sum[row[0]] / d_answered_correctly_u_count[row[0]]
                explanation_u_avg[num] = d_explanation_u_sum[row[0]] / d_answered_correctly_u_count[row[0]]
            else:
                answered_correctly_u_avg[num] = np.nan
                elapsed_time_u_avg[num] = np.nan
                explanation_u_avg[num] = np.nan

            if len(d_timestamp_u[row[0]]) == 0:
                timestamp_u_recency_1[num] = np.nan
                timestamp_u_recency_2[num] = np.nan
                timestamp_u_recency_3[num] = np.nan
            elif len(d_timestamp_u[row[0]]) == 1:
                timestamp_u_recency_1[num] = row[5] - d_timestamp_u[row[0]][0]
                timestamp_u_recency_2[num] = np.nan
                timestamp_u_recency_3[num] = np.nan
            elif len(d_timestamp_u[row[0]]) == 2:
                timestamp_u_recency_1[num] = row[5] - d_timestamp_u[row[0]][1]
                timestamp_u_recency_2[num] = row[5] - d_timestamp_u[row[0]][0]
                timestamp_u_recency_3[num] = np.nan
            elif len(d_timestamp_u[row[0]]) == 3:
                timestamp_u_recency_1[num] = row[5] - d_timestamp_u[row[0]][2]
                timestamp_u_recency_2[num] = row[5] - d_timestamp_u[row[0]][1]
                timestamp_u_recency_3[num] = row[5] - d_timestamp_u[row[0]][0]

            if len(d_timestamp_u_incorrect[row[0]]) == 0:
                timestamp_u_incorrect_recency[num] = np.nan
            else:
                timestamp_u_incorrect_recency[num] = row[5] - d_timestamp_u_incorrect[row[0]][0]

            # prev10 and prev5 will behave a little differently in the beginning
            if len(d_answered_correctly_prev10_u[row[0]]) == 0:
                answered_correctly_prev10_u_avg[num] = np.nan
            else:
                answered_correctly_prev10_u_avg[num] = sum(d_answered_correctly_prev10_u[row[0]])/\
                                                        len(d_answered_correctly_prev10_u[row[0]])
            if len(d_answered_correctly_prev10_u[row[0]]) > 5:
                answered_correctly_prev5_u_avg[num] = sum(d_answered_correctly_prev10_u[row[0]][-5:])/5
            else:
                answered_correctly_prev5_u_avg[num] = np.nan
                
            # Lecture assignation
            lectures_u_count[num] = d_lectures_u_count[row[0]]

            # ------------------------------------------------------------------
            # Question features assignation
            if d_answered_correctly_q_count[row[2]] != 0:
                answered_correctly_q_avg[num] = d_answered_correctly_q_sum[row[2]] / d_answered_correctly_q_count[row[2]]
                elapsed_time_q_avg[num] = d_elapsed_time_q_sum[row[2]] / d_answered_correctly_q_count[row[2]]
                explanation_q_avg[num] = d_explanation_ac_q_count[row[2]] / d_answered_correctly_q_count[row[2]]
            else:
                answered_correctly_q_avg[num] = np.nan
                elapsed_time_q_avg[num] = np.nan
                explanation_q_avg[num] = np.nan

            # ------------------------------------------------------------------
            # Client Question assignation
            answered_correctly_uq_count[num] = d_answered_correctly_uq[row[0]][row[2]]

            # ------------------------------------------------------------------
            # Client Part assignation
            answered_correctly_up_count[num] = d_answered_correctly_up_count[row[0]][row[6]]
            if d_answered_correctly_up_count[row[0]][row[6]] == 0:
                answered_correctly_up_avg[num] = np.nan
            else:
                answered_correctly_up_avg[num] = d_answered_correctly_up_sum[row[0]][row[6]] /\
                                                    d_answered_correctly_up_count[row[0]][row[6]]
            # ------------------------------------------------------------------
            # Client Community assignation
            answered_correctly_uc_count[num] = d_answered_correctly_uc_count[row[0]][row[7]]
            if d_answered_correctly_uc_count[row[0]][row[7]] == 0:
                answered_correctly_uc_avg[num] = np.nan
            else:
                answered_correctly_uc_avg[num] = d_answered_correctly_uc_sum[row[0]][row[7]] /\
                                                    d_answered_correctly_uc_count[row[0]][row[7]]  

            # ------------------------------------------------------------------
            # Client Tag assignation
            for t in row[8]:
                answered_correctly_ut_cum_count[num] += d_answered_correctly_ut_count[row[0]][t]
                answered_correctly_ut_cum_sum[num] += d_answered_correctly_ut_sum[row[0]][t]

            if answered_correctly_ut_cum_count[num] == 0:
                answered_correctly_ut_cum_avg[num] = np.nan
            else:
                answered_correctly_ut_cum_avg[num] = answered_correctly_ut_cum_sum[num] /\
                                                        answered_correctly_ut_cum_count[num]   
                
            # ------------------------------------------------------------------
            # Client question assignation
            
            # If this is a question we have engineered
            if row[2] in cond_probs['content_id_keys'].keys():
                
                # If the user has answered the corresponding previous question
                tmp_idx = cond_probs['content_id_keys'][row[2]]
                tmp_prev_id = cond_probs['content_id_prev'][tmp_idx]
                if tmp_prev_id in d_uq_prev[row[0]].keys():
                    
                    # Assign the associated probability
                    tmp_prev_ac = d_uq_prev[row[0]][tmp_prev_id]
                    if tmp_prev_ac == 0:
                        cond_uq_prob[num] = cond_probs['prev_wrong_current_mean'][tmp_idx]
                    elif tmp_prev_ac == 1:
                        cond_uq_prob[num] = cond_probs['prev_right_current_mean'][tmp_idx]
                    elif tmp_prev_ac == -1:
                        cond_uq_prob[num] = np.nan
                        print('Unexpected output in Client Question Assignation')
                        print(tmp_idx)
                        print(tmp_prev_id)
                        print(tmp_prev_ac)                                            
                else:
                    cond_uq_prob[num] = np.nan
            else:
                cond_uq_prob[num] = np.nan
                

            # ------------------------------------------------------------------
            # ------------------------------------------------------------------
            # Client features updates
            d_answered_correctly_u_count[row[0]] += 1
            d_elapsed_time_u_sum[row[0]] += row[3]
            d_explanation_u_sum[row[0]] += int(row[4])
            if len(d_timestamp_u[row[0]]) == 3:
                d_timestamp_u[row[0]].pop(0)
            d_timestamp_u[row[0]].append(row[5])
            # ------------------------------------------------------------------
            # Question features updates
            d_answered_correctly_q_count[row[2]] += 1
            d_elapsed_time_q_sum[row[2]] += row[3]
            d_explanation_ac_q_count[row[2]] += int(row[4])
            # ------------------------------------------------------------------
            # Client Question updates
            d_answered_correctly_uq[row[0]][row[2]] += 1
            # ------------------------------------------------------------------
            # Client Part updates
            d_answered_correctly_up_count[row[0]][row[6]] += 1 
            # ------------------------------------------------------------------
            # Client Community updates
            d_answered_correctly_uc_count[row[0]][row[7]] += 1    
            # ------------------------------------------------------------------
            # Client Tag updates
            for t in row[8]:
                d_answered_correctly_ut_count[row[0]][t] += 1            

            # ------------------------------------------------------------------
            # Flag for training and inference. Note: this is almost identical to update_features()
            if not is_inference:
                # ------------------------------------------------------------------
                # Client question updates
                if row[2] in cond_probs['content_id_prev']:
                    if d_uq_prev[row[0]][row[2]] == -1: # User has not answered question                    
                        d_uq_prev[row[0]][row[2]] = row[1]

                # ------------------------------------------------------------------
                # Client features updates
                d_answered_correctly_u_sum[row[0]] += row[1]
                if row[1] == 0:
                    if len(d_timestamp_u_incorrect[row[0]]) == 1:
                        d_timestamp_u_incorrect[row[0]].pop(0)
                    d_timestamp_u_incorrect[row[0]].append(row[5])

                if len(d_answered_correctly_prev10_u[row[0]]) == 10:
                    d_answered_correctly_prev10_u[row[0]].pop(0)
                d_answered_correctly_prev10_u[row[0]].append(row[1])                

                # ------------------------------------------------------------------
                # Question features updates
                d_answered_correctly_q_sum[row[2]] += row[1]

                # ------------------------------------------------------------------
                # Client Part features updates
                d_answered_correctly_up_sum[row[0]][row[6]] += row[1]

                # ------------------------------------------------------------------
                # Client Community features updates
                d_answered_correctly_uc_sum[row[0]][row[7]] += row[1]
                
                # ------------------------------------------------------------------
                # Client Community tag updates                
                for t in row[8]:
                    d_answered_correctly_ut_sum[row[0]][t] += row[1]                 

                # ------------------------------------------------------------------
             
    # END FOR LOOP
    
    user_df = pd.DataFrame({'answered_correctly_u_avg': answered_correctly_u_avg
                            ,'elapsed_time_u_avg': elapsed_time_u_avg
                            ,'explanation_u_avg': explanation_u_avg
                            ,'answered_correctly_q_avg': answered_correctly_q_avg
                            ,'answered_correctly_prev10_u_avg': answered_correctly_prev10_u_avg
                            ,'answered_correctly_prev5_u_avg': answered_correctly_prev5_u_avg
                            ,'elapsed_time_q_avg': elapsed_time_q_avg
                            ,'explanation_q_avg': explanation_q_avg
                            ,'answered_correctly_uq_count': answered_correctly_uq_count
                            ,'timestamp_u_recency_1': timestamp_u_recency_1
                            ,'timestamp_u_recency_2': timestamp_u_recency_2
                            ,'timestamp_u_recency_3': timestamp_u_recency_3
                            ,'timestamp_u_incorrect_recency': timestamp_u_incorrect_recency
                            ,'answered_correctly_up_avg' : answered_correctly_up_avg
                            ,'answered_correctly_up_count' : answered_correctly_up_count
                            ,'answered_correctly_uc_avg' : answered_correctly_uc_avg
                            ,'answered_correctly_uc_count' : answered_correctly_uc_count
                            ,'answered_correctly_ut_cum_count' : answered_correctly_ut_cum_count 
                            ,'answered_correctly_ut_cum_avg' : answered_correctly_ut_cum_avg
                            ,'lectures_u_count' : lectures_u_count
                            ,'cond_uq_prob' : cond_uq_prob
                           })
    
    # Only need to concat rows if building a model or inferring
    if (not KAGGLE_RUN) or is_inference:
        df = pd.concat([df, user_df], axis = 1)
        
    # Return
    return df
        
def update_features(df, features_dicts, cond_probs):
    # -----------------------------------------------------------------------
    # Get feature dicts
    d_answered_correctly_u_sum = features_dicts['d_answered_correctly_u_sum']
    d_answered_correctly_q_sum = features_dicts['d_answered_correctly_q_sum']
    d_timestamp_u_incorrect = features_dicts['d_timestamp_u_incorrect']
    d_answered_correctly_prev10_u = features_dicts['d_answered_correctly_prev10_u']
    d_answered_correctly_up_sum = features_dicts['d_answered_correctly_up_sum']
    d_answered_correctly_uc_sum = features_dicts['d_answered_correctly_uc_sum']
    d_answered_correctly_ut_sum = features_dicts['d_answered_correctly_ut_sum']
    d_uq_prev = features_dicts['d_uq_prev']
    
    for row in df[['user_id'
                   ,'answered_correctly'
                   ,'content_id'
                   ,'content_type_id'
                   ,'timestamp'
                   ,'part'
                   ,'community'
                   ,'tags' # 7
                  ]].values:
        if row[3] == 0:
            # ------------------------------------------------------------------
            # Client question updates
            if row[2] in cond_probs['content_id_prev']:
                if d_uq_prev[row[0]][row[2]] == -1: # User has not answered question                    
                    d_uq_prev[row[0]][row[2]] = row[1]
            # ------------------------------------------------------------------
            # Client features updates
            d_answered_correctly_u_sum[row[0]] += row[1]
            if row[1] == 0:
                if len(d_timestamp_u_incorrect[row[0]]) == 1:
                    d_timestamp_u_incorrect[row[0]].pop(0)
                d_timestamp_u_incorrect[row[0]].append(row[4])
                
            if len(d_answered_correctly_prev10_u[row[0]]) == 10:
                d_answered_correctly_prev10_u[row[0]].pop(0)
            d_answered_correctly_prev10_u[row[0]].append(row[1])                   
            # ------------------------------------------------------------------
            # Question features updates
            d_answered_correctly_q_sum[row[2]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Part features updates
            d_answered_correctly_up_sum[row[0]][row[5]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Community features updates
            d_answered_correctly_uc_sum[row[0]][row[6]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Community tag updates  
            for t in row[7]:
                d_answered_correctly_ut_sum[row[0]][t] += row[1]               

def read_and_preprocess(KAGGLE_RUN):

    # Make feature_dicts dictionary for easy passing between functions
    features_dicts = {
        # Client dictionaries
        'd_answered_correctly_u_count': defaultdict(int)
        ,'d_answered_correctly_u_sum': defaultdict(int)
        ,'d_answered_correctly_prev10_u': defaultdict(list)
        ,'d_elapsed_time_u_sum': defaultdict(int)
        ,'d_explanation_u_sum': defaultdict(int)
        ,'d_timestamp_u': defaultdict(list)
        ,'d_timestamp_u_incorrect': defaultdict(list)
        ,'d_lectures_u_count': defaultdict(int)
        
        # Question Dictionaries
        ,'d_answered_correctly_q_count': defaultdict(int)
        ,'d_answered_correctly_q_sum': defaultdict(int)
        ,'d_elapsed_time_q_sum': defaultdict(int)
        ,'d_explanation_ac_q_count': defaultdict(int)
        
        # Client Question dictionary (how many times has the client seen the question)
        ,'d_answered_correctly_uq': defaultdict(lambda: defaultdict(int))
        
        # Client Part dictionary
        ,'d_answered_correctly_up_count': defaultdict(lambda: defaultdict(int))
        ,'d_answered_correctly_up_sum': defaultdict(lambda: defaultdict(int))
        
        # Client Community Dictionary
        ,'d_answered_correctly_uc_count': defaultdict(lambda: defaultdict(int))
        ,'d_answered_correctly_uc_sum' : defaultdict(lambda: defaultdict(int))
        
        # Client Tag Dictionary
        ,'d_answered_correctly_ut_count': defaultdict(lambda: defaultdict(int))
        ,'d_answered_correctly_ut_sum': defaultdict(lambda: defaultdict(int))
        
        # Previous questions dictionaries
        ,'d_uq_prev' : defaultdict(lambda: defaultdict(lambda: int(-1))) #{user_id : {question_id : ac}}
    }
    
    # Load questions and cond_probs
    questions_df = get_questions(KAGGLE_RUN)
    cond_probs = get_cond_probs(KAGGLE_RUN)
        
    # Need to load all columns when batching
    train_cols = ['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
               'task_container_id', 'user_answer', 'answered_correctly',
               'prior_question_elapsed_time', 'prior_question_had_explanation']
    
    # Columns we need
    feld_needed = ['timestamp'
                   ,'user_id'
                   ,'answered_correctly'
                   ,'content_id'
                   ,'content_type_id'
                   ,'prior_question_elapsed_time'
                   ,'prior_question_had_explanation'
                  ]
    total_train_count = 0
    for i in range(1):
        if not KAGGLE_RUN:
            train = dt.fread('./input/train.csv'
                            ,max_nrows = 1 * 1000000
                            ).to_pandas()
            
        else:
            train = dt.fread('../input/riiid-test-answer-prediction/train.csv'
                            ,skip_to_line = i * 10000000 + 2
                            ,max_nrows = 10000000
                            ).to_pandas()
            

        # Needed when skipping rows
        train.columns = train_cols
        train = train[feld_needed]
        
        # Record total train count (to ensure using all rows when batching on kaggle)
        total_train_count += train.shape[0]
    

        # No need for validation set if running on kaggle
        if False:
            pass
        else:
            valid = None

        # Changing dtype to avoid lightgbm error
        train['prior_question_had_explanation'] = train.prior_question_had_explanation.fillna(False).astype('int8')

        # Merge with questions dataframe
        q_cols = ['question_id', 'part', 'community', 'correct_answer', 'tags', 'n_tags'
                 ,'tag_1', 'tag_2', 'tag_3']
        train = pd.merge(train, questions_df[q_cols]
                         ,left_on = 'content_id'
                         ,right_on = 'question_id'
                         ,how = 'left')

        print('Train Feature Calculation Started - ' + str(i))
        train = add_features(train, features_dicts, cond_probs, KAGGLE_RUN = KAGGLE_RUN, is_inference = False)
        
        # Only run once for model buildling
        if not KAGGLE_RUN:
            break
    
    print('Number of training rows read: ' + str(total_train_count))
    
    # Filter by content_type_id to discard lectures for modeling
    if not KAGGLE_RUN:
        train = train.loc[train.content_type_id == 0].reset_index(drop = True) 
    
    return train, valid, questions_df, features_dicts



In [ ]:
train, valid, questions_df, features_dicts = read_and_preprocess(False)

In [ ]:
train